# TextAttack End-to-End

This tutorial provides a broad end-to-end overview of training, evaluating, and attacking a model using TextAttack.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/0_End_to_End.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/0_End_to_End.ipynb)

In [1]:
!pip3 install textattack[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 6.5 MB/s 
     |████████████████████████████████| 365 kB 67.4 MB/s 
     |████████████████████████████████| 5.5 MB 55.8 MB/s 
     |████████████████████████████████| 60 kB 9.3 MB/s 
     |████████████████████████████████| 125 kB 62.3 MB/s 
     |████████████████████████████████| 41.4 MB 1.3 MB/s 
     |████████████████████████████████| 3.6 MB 40.6 MB/s 
     |████████████████████████████████| 401 kB 53.7 MB/s 
     |████████████████████████████████| 769 kB 61.2 MB/s 
     |████████████████████████████████| 511.7 MB 5.8 kB/s 
     |████████████████████████████████| 125 kB 70.7 MB/s 
     |████████████████████████████████| 5.9 MB 47.8 MB/s 
     |████████████████████████████████| 212 kB 66.3 MB/s 
     |████████████████████████████████| 115 kB 79.8 MB/s 
     |████████████████████████████████| 163 kB 68.0 MB/s 
     |███████████████████████████

## Training

First, we're going to train a model. TextAttack integrates directly with [transformers](https://github.com/huggingface/transformers/) and [datasets](https://github.com/huggingface/datasets) to train any of the `transformers` pre-trained models on datasets from `datasets`. 

Let's use the Rotten Tomatoes Movie Review dataset: it's relatively short , and showcasesthe key features of `textattack train`. Let's take a look at the dataset using `textattack peek-dataset`:

In [2]:
!textattack peek-dataset --dataset-from-huggingface imdb

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.
100% 3/3 [00:00<00:00, 343.11it/s]
textattack: Loading datasets dataset im

The dataset looks good! It's lowercased already, so we'll make sure our model is uncased. The longest input is 51 words, so we can cap our maximum sequence length (`--model-max-length`) at 64.

We'll train [`distilbert-base-uncased`](https://huggingface.co/transformers/model_doc/distilbert.html), since it's a relatively small model, and a good example of how we integrate with `transformers`.

So we have our command:

```bash
textattack train                      \ # Train a model with TextAttack
    --model distilbert-base-uncased   \ # Using distilbert, uncased version, from `transformers`
    --dataset rotten_tomatoes         \ # On the Rotten Tomatoes dataset
    --model-num-labels 3              \ # That has 2 labels
    --model-max-length 64             \ # With a maximum sequence length of 64
    --per-device-train-batch-size 128 \ # And batch size of 128
    --num-epochs 3                    \ # For 3 epochs 
```

Now let's run it (please remember to use GPU if you have access):

In [8]:
!textattack train --model distilroberta-base --dataset imdb --model-num-labels 2 --model-max-length 2480 --per-device-train-batch-size 32 --num-epochs 3

textattack: Loading transformers AutoModelForSequenceClassification: distilroberta-base
Downloading: 100% 480/480 [00:00<00:00, 463kB/s]
Downloading: 100% 331M/331M [00:04<00:00, 70.4MB/s]
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClass

## Evaluation

We successfully fine-tuned `distilbert-base-cased` for 3 epochs. Now let's evaluate it using `textattack eval`. This is as simple as providing the path to the pretrained model (that you just obtain from running the above command!) to `--model`, along with the number of evaluation samples. `textattack eval` will automatically load the evaluation data from training:

In [12]:
!textattack eval --num-examples 1000 --model ./outputs/2022-11-12-19-44-01-289961/best_model/ --dataset-from-huggingface imdb --dataset-split test

100% 3/3 [00:00<00:00, 868.87it/s]
textattack: Loading datasets dataset imdb, split test.
textattack: Got 1000 predictions.
textattack: Correct 936/1000 (93.60%)


Awesome -- we were able to train a model up to 84.9% accuracy on the test dataset – with only a single command!

## Attack

Finally, let's attack our pre-trained model. We can do this the same way as before (by providing the path to the pretrained model to `--model`). For our attack, let's use the "TextFooler" attack recipe, from the paper ["Is BERT Really Robust? A Strong Baseline for Natural Language Attack on Text Classification and Entailment" (Jin et al, 2019)](https://arxiv.org/abs/1907.11932). We can do this by passing `--recipe textfooler` to `textattack attack`.

> *Warning*: We're printing out 100 examples and, if the attack succeeds, their perturbations. The output of this command is going to be quite long!


In [13]:
!textattack attack --recipe textfooler --num-examples 100 --model ./outputs/2022-11-12-19-44-01-289961/best_model/ --dataset-from-huggingface imdb --dataset-split test

100% 3/3 [00:00<00:00, 470.48it/s]
textattack: Loading datasets dataset imdb, split test.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100% 481M/481M [00:12<00:00, 39.5MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpu_2v4wig.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0

Looks like our model was 84% successful (makes sense - same evaluation set as `textattack eval`!), meaning that TextAttack attacked the model with 84 examples (since the attack won't run if an example is originally mispredicted). The attack success rate was 98.8%, meaning that TextFooler failed to find an adversarial example only 1.2% (1 out of 84) of the time.


## Conclusion

That's all, folks! We've learned how to train, evaluate, and attack a model with TextAttack, using only three commands! 😀